In [219]:
import cPickle as pickle
from ast import literal_eval
from sklearn.cross_validation import StratifiedShuffleSplit
import numpy as np
import json
import pandas as pd

In [157]:
with open('pickled/userdata.pkl', 'r') as picklefile:
    alldata = pickle.load(picklefile)

In [159]:
data = alldata[['user_id', 'is_elite', 'friends', 'friends_num']]

In [188]:
split = StratifiedShuffleSplit(data.is_elite, n_iter = 1, test_size = 0.001, random_state = 0)

In [189]:
for train_index, test_index in split: 
    data75 = data.loc[train_index,:]
    data25 = data.loc[test_index,:]

In [288]:
nodes = data[['user_id', 'is_elite', 'friends_num']]

In [289]:
nodes.columns = ['name', 'group', 'friends']

In [290]:
nodes.group.replace((1, 0), (1, 2), inplace = True)

In [305]:
nodes_dict = [ 
    dict([
        (colname, row[i]) 
        for i,colname in enumerate(nodes.columns)
    ])
    for row in nodes.values
]

In [304]:
d

[{'friends': 202, 'group': 1, 'name': '18kPq7GPye-YQ3LyKyAZPw'},
 {'friends': 1936, 'group': 1, 'name': 'rpOyqD_893cqmDAtJLbdog'},
 {'friends': 405, 'group': 1, 'name': '4U9kSBLuBDU391x6bxU-YA'},
 {'friends': 4, 'group': 2, 'name': 'fHtTaujcyKvXglE33Z5yIw'},
 {'friends': 4, 'group': 1, 'name': 'SIBCL7HBkrP4llolm4SC2A'},
 {'friends': 1193, 'group': 1, 'name': '8J4IIYcqBlFch8T90N923A'},
 {'friends': 38, 'group': 1, 'name': 'ysYmC-ufbdmVEX9yAv-VEQ'},
 {'friends': 2, 'group': 1, 'name': 'WPOKvkacSKHx_bIG1alFiA'},
 {'friends': 75, 'group': 2, 'name': 'UTS9XcT14H2ZscRIf0MYHQ'},
 {'friends': 6, 'group': 2, 'name': 'qL7Astun3i7qwr2IL5iowA'},
 {'friends': 126, 'group': 1, 'name': 'ZWOj6LmzwGvMDh-A85EOtA'},
 {'friends': 14, 'group': 2, 'name': 'HDQixQ-WZEV0LVPJlIGQeQ'},
 {'friends': 17, 'group': 1, 'name': 'za5Q2uYrTmky9w_3h9k_wQ'},
 {'friends': 0, 'group': 2, 'name': 'HI-QmpsYxP5AJtRPanXKQw'},
 {'friends': 5, 'group': 2, 'name': 'HZJ9qzZlXC8xeC8v7VT8WA'},
 {'friends': 1, 'group': 2, 'name': 'y6

In [206]:
nodes.head()

,name,group,friends
122901,XBsTe6_a_zzH_JiIrR6XSw,2,1
319782,kO84JHf17iFnHtO6aN57rA,2,0
161762,jIvz8ZpwC1xpLIbB8oT1fQ,2,2
180867,xEEbMSm-c_TP1A2UcQnYtg,2,0
468707,S85GH_oaZ9NJJfpI0Mk7_g,2,0


In [292]:
with open('names_all.json', 'w') as outfile:
    json.dump(d, outfile, indent = 1)

### edges

In [293]:
# get user indices
nodes['ind'] = range(nodes.shape[0])
ind = nodes[['name', 'ind']].set_index('name')
ind_dict = ind['ind'].to_dict()

In [295]:
# create edges df
edges = data[['user_id', 'friends']]

In [296]:
edges_tst = edges

In [297]:
edges_tst.friends = edges_tst.friends.apply(lambda x: ', '.join(x))

In [224]:
# edges_tst = edges_tst[edges_tst.friends != '']

In [298]:
working = pd.concat([pd.Series(row['user_id'], row['friends'].split(', '))              
                    for _, row in edges_tst.iterrows()]).reset_index()

In [299]:
working.columns = ['friend', 'user_id']

In [300]:
working = working[working.friend != '']

In [301]:
working['user_id_ind'] = working.user_id.apply(lambda x: ind_dict[x])

In [287]:
ind_dict['pinsMHhHigtYkEXvNR9urw']

KeyError: 'pinsMHhHigtYkEXvNR9urw'

In [302]:
working['friend_ind'] = working.friend.apply(lambda x: ind_dict[x])

In [306]:
links = working[['user_id_ind', 'friend_ind']]

In [308]:
links.columns = ['source', 'target']

In [309]:
links['value'] = 1

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [311]:
links_dict = [ 
    dict([
        (colname, row[i]) 
        for i,colname in enumerate(links.columns)
    ])
    for row in links.values
]

In [314]:
combined = {"nodes" : nodes_dict, "links" : links_dict}

In [315]:
with open('yelpdata.json', 'w') as outfile:
    json.dump(combined, outfile, indent = 1)

In [319]:
len(links_dict)

3563817

In [317]:
combined['links']

[{'source': 0, 'target': 1, 'value': 1},
 {'source': 0, 'target': 2, 'value': 1},
 {'source': 0, 'target': 3, 'value': 1},
 {'source': 0, 'target': 5, 'value': 1},
 {'source': 0, 'target': 90, 'value': 1},
 {'source': 0, 'target': 11, 'value': 1},
 {'source': 0, 'target': 180, 'value': 1},
 {'source': 0, 'target': 728, 'value': 1},
 {'source': 0, 'target': 1647, 'value': 1},
 {'source': 0, 'target': 2190, 'value': 1},
 {'source': 0, 'target': 2191, 'value': 1},
 {'source': 0, 'target': 2377, 'value': 1},
 {'source': 0, 'target': 2466, 'value': 1},
 {'source': 0, 'target': 2470, 'value': 1},
 {'source': 0, 'target': 3768, 'value': 1},
 {'source': 0, 'target': 3769, 'value': 1},
 {'source': 0, 'target': 3944, 'value': 1},
 {'source': 0, 'target': 4035, 'value': 1},
 {'source': 0, 'target': 26658, 'value': 1},
 {'source': 0, 'target': 4580, 'value': 1},
 {'source': 0, 'target': 4762, 'value': 1},
 {'source': 0, 'target': 4767, 'value': 1},
 {'source': 0, 'target': 4768, 'value': 1},
 {'so

In [241]:
working['friend_ind'] = 

,friend,user_id,user_id_ind
0,rO7wrC6TDB8t6a93nQdIlg,XBsTe6_a_zzH_JiIrR6XSw,0
1,,kO84JHf17iFnHtO6aN57rA,1
2,pinsMHhHigtYkEXvNR9urw,jIvz8ZpwC1xpLIbB8oT1fQ,2
3,RH7C3Akb_pl7vq0WXM66Hw,jIvz8ZpwC1xpLIbB8oT1fQ,3
4,,xEEbMSm-c_TP1A2UcQnYtg,4


In [ ]:
def add_edges(row):
    if len(row.friends) > 0:
        

In [208]:
edges_test.friends.apply()

,user_id,friends
122901,XBsTe6_a_zzH_JiIrR6XSw,[rO7wrC6TDB8t6a93nQdIlg]
319782,kO84JHf17iFnHtO6aN57rA,[]
161762,jIvz8ZpwC1xpLIbB8oT1fQ,"[pinsMHhHigtYkEXvNR9urw, RH7C3Akb_pl7vq0WXM66Hw]"
180867,xEEbMSm-c_TP1A2UcQnYtg,[]
468707,S85GH_oaZ9NJJfpI0Mk7_g,[]
144584,CGYWxwO4WbvaSyIEQH56eQ,[]
121510,UJOr4VE9pKlFsi_oab-l2Q,[]
265196,Y44SkN5ozanM0nWSEUg5eg,[]
191246,uvyUAwm3uY-GlUdseNK-hA,"[osmdTGbwUJeV6ihgmtBycg, LNGdchsYlSQpvwNw3CrH8..."
86519,gsawF6gPO-NASUmHgesC1w,[]


In [141]:
edges.friends = edges.friends.apply(lambda x: x[0] if len(x) > 0 else '')

In [142]:
edges.columns = ['Source', 'Target']

In [143]:
edges = edges[edges.Target != '']

In [144]:
edges['Type'] = 'Undirected'

In [145]:
edges.to_csv('user_edges5.csv', index = False)

In [149]:
edges

,Source,Target,Type
295033,ltxy8v4MoSONtTwveOPllw,5pTfYMR4umwPaXbslrDR-w,Undirected
318776,D4EOVMnGLfMTsdU9vBq5Nw,fNDTgzEWE_RCGvECN-kHGQ,Undirected
332508,Tajebaj04htTYn6D68r8ug,bz6CkTf3XFfBc3GY66QBiA,Undirected
367546,B8fJNu3eeaXi7285f0KbXQ,KdDaH2G9d4ggaQiagjYMKg,Undirected
518985,qlWODdKjmgqip_JsaA7Tcg,dCw3yWFaYYOTwncGPJO3Ig,Undirected
344844,vcm9ulAN-Sb6AI-1TGDf7g,1I2OMYsGhCdMzg4oCvgo6g,Undirected
217024,73SMgkYd2sWnhS2yKxCoKg,0kHlIKnWzaMQ1TLYQB1rPg,Undirected
542008,rRLVdvpQ9wnCZIEdkkENzw,u-w6YX5FqqgumUWCNPjZ9A,Undirected
90045,wtetKGea0cE6J4iE6GK5Vw,i63u3SdbrLsP4FxiSKP0Zw,Undirected
420183,_MdGXJe0azoWflTyERmnJQ,YDoojZldnPFe3lrNy1aRCA,Undirected
